## Imports 

In [1]:
import numpy as np
import os
import random
from random import choice 
from copy import deepcopy

## Reading databases

In [2]:
def read_databases(filename):
    x1 = []
    x2 = []
    y = []
    for line in open(filename):
        csv_row = line.strip().split(',')
        csv_row = [float(number) for number in csv_row]
        x1.append(csv_row[0])
        x2.append(csv_row[1])
        y.append(csv_row[2])
    return x1, x2, y

In [3]:
filename = 'datasets/synth1/synth1-test.csv'
x1_train, x2_train, y_train = read_databases(filename)

In [4]:
filename = 'datasets/synth1/synth1-train.csv'
x1_test, x2_test, y_test = read_databases(filename)

## Defining individual class

In [5]:
expression = [['expression', 'op', 'expression'], 'variable', 'value']
operation = ['+', '-', '*']
variable = ['x1', 'x2']
value = ['random']

In [6]:
terminals_dict = dict()
terminals_dict['op'] = operation
terminals_dict['variable'] = variable
terminals_dict['value'] = value

In [7]:
class Node(object):
    def __init__(self, data='op', level=0):
        self.data = data
        self.type = data
        self.left_child = None
        self.right_child = None
        self.level = level
        self.max_level = 7
        self.terminal = False
        self._fitness = None

    def initial_node(self):
        self.add_left('expression')
        self.add_right('expression')
    
    def initialize(self):
        self.add_left('expression')
        self.add_right('expression')
        self.expand_children()
        self.get_parameters()

    def add_left(self, value):
        self.left_child = Node(value, level=self.level+1)
    
    def add_right(self, value):
        self.right_child = Node(value, level=self.level+1)
        
    def show(self):
        print(self.level, ": " , self.data)
        if self.left_child:
            self.left_child.show()
        
        if self.right_child:
            self.right_child.show()
    
    def get_expression(self):
        if self.data == 'expression':
            self.data = choice(expression)
            if type(self.data) == list:
                if self.level < self.max_level: 
                    self.add_left(self.data[0])
                    self.add_right(self.data[2])
                    self.data = self.data[1]
                else:
                    self.data = 'value'
                    
        if self.data != 'op':
            self.terminal = True
        else:
            self.terminal = False
                    
        if self.left_child is not None:
            self.left_child.get_expression()
        
        if self.right_child is not None:
            self.right_child.get_expression()
        
        self.type = self.data
    
    def expand_children(self):
        self.left_child.get_expression()
        self.right_child.get_expression()
    
    def get_parameters(self):
        self.data = choice(terminals_dict[self.data])
        if self.data == 'random':
            self.data = random.random()*5
        
        if self.left_child is not None:
            self.left_child.get_parameters()
        
        if self.right_child is not None:
            self.right_child.get_parameters()
    
    def evaluate(self, x1=0, x2=0):
        
        if self.terminal:
            if self.data == 'x1':
                return x1
            elif self.data == 'x2':
                return x2
            else:
                return self.data
            
        if self.left_child is not None:
            result_left = self.left_child.evaluate(x1, x2)
        if self.right_child is not None:
            result_right = self.right_child.evaluate(x1, x2)
        
        if self.data == '+':
            return result_left + result_right
        elif self.data == '-':
            return result_left - result_right
        elif self.data == '*':
            return result_left * result_right
    
    def error(self, x1, x2, y):
        return (y - self.evaluate(x1, x2))**2
    
    def fitness(self, x1_list, x2_list, y_list):
        
        if self._fitness == None:
            error_sum = 0
            for x1, x2, y in zip(x1_list, x2_list, y_list):
                error_sum += self.error(x1, x2, y)
            y_mean = y_list - np.mean(y_list)
            y_mean_sum = sum(y_mean**2)
            self._fitness = (error_sum/y_mean_sum)**0.5
            
        return self._fitness
    
    def change_node(self):
        self.data = choice(terminals_dict[self.type])
        if self.data == 'random':
            self.data = random.random()*5
    
    def mutate(self, m_level):
        if m_level == self.level:
            return self.change_node()
        elif np.random.rand() < 0.5 and self.left_child is not None:
            self.left_child.mutate(m_level)
        elif self.right_child is not None:
            self.right_child.mutate(m_level)
        else:
            return self.change_node()
    
    def subtree_mutate(self, m_level):
        if m_level == self.level:
            self.data = 'op'
            self.type = 'op'
            self.terminal = False
            self.add_left('expression')
            self.add_right('expression')
            self.get_expression()
            self.get_parameters()
            return self
        elif np.random.rand() < 0.5 and self.left_child is not None:
            self.left_child.subtree_mutate(m_level)
        elif self.right_child is not None:
            self.right_child.subtree_mutate(m_level)
        else:
            self.data = 'op'
            self.type = 'op'
            self.terminal = False
            self.add_left('expression')
            self.add_right('expression')
            self.get_expression()
            self.get_parameters()
            return self
    
    def subtree(self, directions_list):
        if len(directions_list) <= 1:
            if directions_list[0] == 'left':
                #return self.left_child
                return self.left_child if self.left_child else self
            elif directions_list[0] == 'right':
                return self.right_child if self.right_child else self
            else:
                raise ValueError('Please give a valid id for tree directions ["left"] or ["right"].', directions_list[0])
        elif directions_list[0] == 'left':
            return self.left_child.subtree(directions_list[1:]) if self.left_child else self
        elif directions_list[0] == 'right':
            return self.right_child.subtree(directions_list[1:]) if self.right_child else self
        else:
            raise ValueError('Please give a valid id for tree directions ["left"] or ["right"].', directions_list[0])
    
    def alter_subtree(self, directions_list, subtree):
        if self.left_child == None:
                self.data = subtree.data
                self.terminal = subtree.terminal
                self.left_child = subtree.left_child
                self.right_child = subtree.right_child
                self.fix_level(self.level)
                #print("Terminal")
        
        elif len(directions_list) == 1:
            if directions_list[0] == 'left':
                self.left_child = subtree
                self.left_child.fix_level(self.level+1)
            elif directions_list[0] == 'right':
                self.right_child = subtree
                self.right_child.fix_level(self.level+1)
            else:
                raise ValueError('Please give a valid id for tree directions ["left"] or ["right"].', directions_list[0])
        
        elif directions_list[0] == 'left':
            if self.left_child:
                self.left_child.alter_subtree(directions_list[1:], subtree)
            else:
                self.left_child = subtree
                self.left_child.fix_level(self.level+1)
        
        elif directions_list[0] == 'right':
            if self.right_child:
                self.right_child.alter_subtree(directions_list[1:], subtree)
            else:
                self.right_child = subtree
                self.right_child.fix_level(self.level+1)
        
        else:
            raise ValueError('Please give a valid id for tree directions ["left"] or ["right"].', directions_list[0])
    
    def gen_directions(self, level):
        directions = ['left', 'right']
        directions_list = []
        for i in range(level):
            directions_list.append(np.random.choice(directions))
        return directions_list
    
    def fix_level(self, level):
        self.level = level
        if self.left_child:
            self.left_child.fix_level(level+1)
        if self.right_child:
            self.right_child.fix_level(level+1)
        
    def crossover(self, parent, crossover_level1, crossover_level2):
        directions_list1 = self.gen_directions(crossover_level1)
        directions_list2 = self.gen_directions(crossover_level2)
        #print(directions_list1, directions_list2)
        subtree1 = deepcopy(self.subtree(directions_list1))
        subtree2 = deepcopy(parent.subtree(directions_list2))
        #subtree1.show()
        #subtree2.show()
        self.alter_subtree(directions_list1, subtree2)
        parent.alter_subtree(directions_list2, subtree1)
        self.fix_level(0)
        parent.fix_level(0)
    
    def tree_deepness(self):
        if self.left_child == None:
            return self.level
        
        else:
            deep = max(self.left_child.tree_deepness(), self.right_child.tree_deepness())
            return deep

In [8]:
root = Node()
root.initial_node()
root.expand_children()
root.show()

root.get_parameters()
print()
root.show()

root.evaluate()

0 :  op
1 :  value
1 :  variable

0 :  +
1 :  3.512969042989135
1 :  x1


3.512969042989135

In [9]:
root.evaluate(2,5)

5.512969042989135

In [10]:
root.error(2,5, 2)

12.340951496999999

## Generating initial population

In [11]:
##Parameters
population_size = 100
crossover_probability = 0.9

In [12]:
class Genetic():
    def __init__(self, population_size, crossover_probability=0.9):
        if crossover_probability < 0 or crossover_probability > 1 :
            raise ValueError('Please give a valid number for crossover probability.')
        self.population_size = population_size
        self.crossover_probability = crossover_probability
        self.mutation_probability = 1 - crossover_probability
        self.population = []
        self.x_values = None
        self.y_values = None
        self.max_level = 7
        for i in range(self.population_size):
            root = Node()
            root.initialize()
            self.population.append(root)
    
    def sort_by_fitness(self):  #, x1_list=self.x_values[0], x2_list=self.x_valus[1], y_list=self.y_values):
        x1_list=self.x_values[0]
        x2_list=self.x_values[1]
        y_list=self.y_values
        self.population.sort(key = lambda x: x.fitness(x1_list, x2_list, y_list))
    
    def rank_selection_pair(self):
        ## Assign probabilities inversely proportional to the position index
        p = list(range(len(solution1.population))) + np.array(1)
        p = np.flipud(p)  # reverse array
        p = p/sum(p)
        parents = np.random.choice(self.population,size=2, p=p)
        p_fitness = [parents[0]._fitness, parents[1]._fitness]
        return (parents, p_fitness) #Return two individuals from population, and their fitness
    
    def tournament(self, k=10):
        selected_index = []
        parents = []
        p_fitness = []
        for i in range(2):
            index = np.random.choice(self.population_size,size=k)
            selected_index.append(max(index, key=lambda p: self.population[p]._fitness))
            parents.append(deepcopy(self.population[selected_index[i]]))
            p_fitness.append(self.population[selected_index[i]]._fitness)
        return (selected_index, parents, p_fitness)
    
    def generate_offspring(self):
        if np.random.rand() < self.crossover_probability:
            ## Crossover
            print("Crossover")
            return self.crossover()
        else:
            ## Mutation
            print("Mutation")
            return self.mutation()
    
    def iteration(self, elitism=False, elitist_operators=False):
        index, parents, p_fitness = self.tournament()
        for parent in parents:
            parent._fitness = None
        
        if np.random.rand() < self.crossover_probability:
            ## Crossover
            print("Crossover")
            children = self.crossover(parents)
        else:
            ## Mutation
            print("Mutation")
            children = self.mutation(parents)
        for child in children:
            child.fitness(*self.x_values, self.y_values)
        #print("Parents fitness: ", p_fitness)
        print()
        #print("Child fitness: ", children[0]._fitness, children[1]._fitness )
        if elitist_operators:
            for i in range(2):
                if children[i]._fitness < p_fitness[i]:
                    self.population[index[i]] = children[i]
                    print("Update!!")
            self.sort_by_fitness()
        else:
            self.population.extend(children)
            self.sort_by_fitness()
            self.population = self.population[:self.population_size]
        
        print("Best: ", self.population[0]._fitness, end="", flush=True)
        print("Average: ", self.population[self.population_size//2]._fitness, end="", flush=True)
        print("Worst: ", self.population[-1]._fitness)
    
    def crossover(self, parents, elitism=False):
        ## Chosing mutation level with a pobability 
        children = [deepcopy(parents[0]), deepcopy(parents[1])]
        #print(children[0].show())
        #print(children[1].show())
        p = list(range(self.max_level)) + np.array(1)
        p = p/sum(p)
        
        crossover_level1 = np.random.choice(self.max_level, p=p) + 1  ## Avoid this number to be zero
        crossover_level2 = np.random.choice(self.max_level, p=p) + 1
        children[0].crossover(children[1], crossover_level1, crossover_level2)
        
        if children[0].tree_deepness() > children[0].max_level or children[1].tree_deepness() > children[1].max_level:
            return self.mutation(parents)
        else:
            return children
    
    def mutation(self, parents, elitism=False):
        children = [deepcopy(parents[0]), deepcopy(parents[1])]
        #print(children[0].show())
        #print(children[1].show())
        p = list(range(self.max_level)) + np.array(1)
        p = p/sum(p)
        for child in children:
            mutation_level = np.random.choice(self.max_level, p=p)
            child.subtree_mutate(mutation_level)
        return children
    
    def evolve(self, generations=100, elitist_operators=True):
        for i in range(generations):
            self.iteration(elitist_operators=elitist_operators)

In [13]:
solution1 = Genetic(population_size, crossover_probability)

In [14]:
solution1.x_values = [x1_test, x2_test]
solution1.y_values = y_test

In [15]:
solution1.sort_by_fitness() #x1_test, x2_test, y_test)

In [16]:
print(solution1.population[0].fitness(x1_test, x2_test, y_test))
solution1.population[0].show()

0.947378087753502
0 :  -
1 :  -
2 :  3.6538407283657586
2 :  x1
1 :  *
2 :  -
3 :  0.2011488701657621
3 :  x1
2 :  x1


In [17]:
print(solution1.population[1].fitness(x1_test, x2_test, y_test))
solution1.population[1].show()

0.9956799337243489
0 :  +
1 :  *
2 :  3.8201579214137937
2 :  2.9940710515824005
1 :  x2


In [18]:
print(solution1.population[2].fitness(x1_test, x2_test, y_test))
solution1.population[2].show()

1.0018519413288824
0 :  *
1 :  1.9709859129152263
1 :  *
2 :  2.3143945129557753
2 :  2.5635936382030655


In [19]:
print(solution1.population[3].fitness(x1_test, x2_test, y_test))
solution1.population[3].show()

1.0154032817565097
0 :  +
1 :  x1
1 :  *
2 :  2.6382204786284147
2 :  4.951806499085806


In [20]:
print(solution1.population[4].fitness(x1_test, x2_test, y_test))
solution1.population[4].show()

1.03081844400484
0 :  +
1 :  4.680593836666182
1 :  -
2 :  4.7104125447277205
2 :  x2


## Selection

In [21]:
solution1.population

In [22]:
solution1.crossover_probability = 0.9
solution1.iteration(elitist_operators=True)

Crossover

Update!!
Best:  0.947378087753502Average:  1.203984688834854Worst:  3.662042521266476


In [23]:
solution1.population[2].show()

0 :  *
1 :  1.9709859129152263
1 :  *
2 :  2.3143945129557753
2 :  2.5635936382030655


In [24]:
solution1.population[2].subtree_mutate(3)

In [25]:
solution1.population[2].show()

0 :  *
1 :  +
2 :  3.3855852535737685
2 :  *
3 :  +
4 :  4.146672037729051
4 :  x1
3 :  x1
1 :  *
2 :  2.3143945129557753
2 :  2.5635936382030655


In [26]:
solution1.population[2].left_child.left_child.terminal

True

In [27]:
solution1.population[2]._fitness = None
solution1.population[2].fitness(x1_test, x2_test, y_test)

3.114227738214444

## Tests area

In [28]:
solution1.evolve(10000)

Crossover

Update!!
Update!!
Best:  0.947378087753502Average:  1.203984688834854Worst:  3.662042521266476
Crossover

Best:  0.947378087753502Average:  1.203984688834854Worst:  3.662042521266476
Crossover

Update!!
Best:  0.947378087753502Average:  1.203984688834854Worst:  3.662042521266476
Crossover

Best:  0.947378087753502Average:  1.203984688834854Worst:  3.662042521266476
Crossover

Best:  0.947378087753502Average:  1.203984688834854Worst:  3.662042521266476
Crossover

Update!!
Best:  0.947378087753502Average:  1.203984688834854Worst:  3.662042521266476
Crossover

Update!!
Update!!
Best:  0.947378087753502Average:  1.203984688834854Worst:  3.662042521266476
Crossover

Best:  0.947378087753502Average:  1.203984688834854Worst:  3.662042521266476
Crossover

Update!!
Best:  0.947378087753502Average:  1.203984688834854Worst:  3.662042521266476
Crossover

Update!!
Best:  0.947378087753502Average:  1.201104973544294Worst:  3.662042521266476
Crossover

Update!!
Update!!
Best:  0.9473780877

Best:  0.7945782294587964Average:  1.1546009260456964Worst:  1.6029188397569334
Crossover

Best:  0.7945782294587964Average:  1.1546009260456964Worst:  1.6029188397569334
Crossover

Best:  0.7945782294587964Average:  1.1546009260456964Worst:  1.6029188397569334
Crossover

Update!!
Best:  0.7945782294587964Average:  1.1545589149836297Worst:  1.6029188397569334
Crossover

Best:  0.7945782294587964Average:  1.1545589149836297Worst:  1.6029188397569334
Crossover

Update!!
Best:  0.7945782294587964Average:  1.1545589149836297Worst:  1.6029188397569334
Mutation

Update!!
Best:  0.7945782294587964Average:  1.1545589149836297Worst:  1.6029188397569334
Mutation

Update!!
Best:  0.7945782294587964Average:  1.1545589149836297Worst:  1.6029188397569334
Crossover

Best:  0.7945782294587964Average:  1.1545589149836297Worst:  1.6029188397569334
Crossover

Update!!
Update!!
Best:  0.7945782294587964Average:  1.1515436444144995Worst:  1.6029188397569334
Crossover

Update!!
Best:  0.7945782294587964Aver

Best:  0.7945782294587964Average:  1.120055866164724Worst:  1.2198739207678528
Crossover

Best:  0.7945782294587964Average:  1.120055866164724Worst:  1.2198739207678528
Crossover

Best:  0.7945782294587964Average:  1.120055866164724Worst:  1.2198739207678528
Crossover

Best:  0.7945782294587964Average:  1.120055866164724Worst:  1.2198739207678528
Crossover

Best:  0.7945782294587964Average:  1.120055866164724Worst:  1.2198739207678528
Crossover

Update!!
Best:  0.7945782294587964Average:  1.120055866164724Worst:  1.2198739207678528
Crossover

Best:  0.7945782294587964Average:  1.120055866164724Worst:  1.2198739207678528
Crossover

Best:  0.7945782294587964Average:  1.120055866164724Worst:  1.2198739207678528
Crossover

Best:  0.7945782294587964Average:  1.120055866164724Worst:  1.2198739207678528
Crossover

Update!!
Best:  0.7945782294587964Average:  1.120055866164724Worst:  1.2198739207678528
Crossover

Best:  0.7945782294587964Average:  1.120055866164724Worst:  1.2198739207678528
Cro

Best:  0.7945782294587964Average:  1.1107013484837054Worst:  1.2134019899750847
Crossover

Update!!
Best:  0.7945782294587964Average:  1.1091636595139718Worst:  1.2134019899750847
Crossover

Best:  0.7945782294587964Average:  1.1091636595139718Worst:  1.2134019899750847
Crossover

Best:  0.7945782294587964Average:  1.1091636595139718Worst:  1.2134019899750847
Crossover

Update!!
Best:  0.7945782294587964Average:  1.1086724094981426Worst:  1.2134019899750847
Crossover

Best:  0.7945782294587964Average:  1.1086724094981426Worst:  1.2134019899750847
Crossover

Best:  0.7945782294587964Average:  1.1086724094981426Worst:  1.2134019899750847
Mutation

Update!!
Best:  0.7945782294587964Average:  1.1086724094981426Worst:  1.2134019899750847
Crossover

Best:  0.7945782294587964Average:  1.1086724094981426Worst:  1.2134019899750847
Crossover

Best:  0.7945782294587964Average:  1.1086724094981426Worst:  1.2134019899750847
Crossover

Best:  0.7945782294587964Average:  1.1086724094981426Worst:  1.2

Best:  0.7945782294587964Average:  1.0951263113117353Worst:  1.2134019899750847
Crossover

Best:  0.7945782294587964Average:  1.0951263113117353Worst:  1.2134019899750847
Crossover

Best:  0.7945782294587964Average:  1.0951263113117353Worst:  1.2134019899750847
Crossover

Best:  0.7945782294587964Average:  1.0951263113117353Worst:  1.2134019899750847
Crossover

Best:  0.7945782294587964Average:  1.0951263113117353Worst:  1.2134019899750847
Crossover

Best:  0.7945782294587964Average:  1.0951263113117353Worst:  1.2134019899750847
Crossover

Best:  0.7945782294587964Average:  1.0951263113117353Worst:  1.2134019899750847
Crossover

Best:  0.7945782294587964Average:  1.0951263113117353Worst:  1.2134019899750847
Crossover

Update!!
Best:  0.7945782294587964Average:  1.0951263113117353Worst:  1.2134019899750847
Crossover

Best:  0.7945782294587964Average:  1.0951263113117353Worst:  1.2134019899750847
Crossover

Update!!
Best:  0.7945782294587964Average:  1.0915945746538012Worst:  1.213401989

...

Best:  0.5196737980620347Average:  0.9470338218451597Worst:  1.0000040011603626
Crossover

Best:  0.5196737980620347Average:  0.9470338218451597Worst:  1.0000040011603626
Crossover

Best:  0.5196737980620347Average:  0.9470338218451597Worst:  1.0000040011603626
Crossover

Best:  0.5196737980620347Average:  0.9470338218451597Worst:  1.0000040011603626
Crossover

Best:  0.5196737980620347Average:  0.9470338218451597Worst:  1.0000040011603626
Crossover

Best:  0.5196737980620347Average:  0.9470338218451597Worst:  1.0000040011603626
Crossover

Best:  0.5196737980620347Average:  0.9470338218451597Worst:  1.0000040011603626
Crossover

Best:  0.5196737980620347Average:  0.9470338218451597Worst:  1.0000040011603626
Crossover

Best:  0.5196737980620347Average:  0.9470338218451597Worst:  1.0000040011603626
Crossover

Best:  0.5196737980620347Average:  0.9470338218451597Worst:  1.0000040011603626
Crossover

Best:  0.5196737980620347Average:  0.9470338218451597Worst:  1.0000040011603626
Crossover


Best:  0.5196737980620347Average:  0.9470338218451597Worst:  1.0000040011603626
Crossover

Best:  0.5196737980620347Average:  0.9470338218451597Worst:  1.0000040011603626
Crossover

Best:  0.5196737980620347Average:  0.9470338218451597Worst:  1.0000040011603626
Crossover

Best:  0.5196737980620347Average:  0.9470338218451597Worst:  1.0000040011603626
Crossover

Best:  0.5196737980620347Average:  0.9470338218451597Worst:  1.0000040011603626
Crossover

Best:  0.5196737980620347Average:  0.9470338218451597Worst:  1.0000040011603626
Crossover

Best:  0.5196737980620347Average:  0.9470338218451597Worst:  1.0000040011603626
Crossover

Best:  0.5196737980620347Average:  0.9470338218451597Worst:  1.0000040011603626
Crossover

Best:  0.5196737980620347Average:  0.9470338218451597Worst:  1.0000040011603626
Crossover

Update!!
Update!!
Best:  0.5196737980620347Average:  0.9470338218451597Worst:  1.0000040011603626
Crossover

Best:  0.5196737980620347Average:  0.9470338218451597Worst:  1.000004001

Best:  0.5196737980620347Average:  0.9456540385080985Worst:  1.0000040011603626
Crossover

Best:  0.5196737980620347Average:  0.9456540385080985Worst:  1.0000040011603626
Crossover

Best:  0.5196737980620347Average:  0.9456540385080985Worst:  1.0000040011603626
Crossover

Best:  0.5196737980620347Average:  0.9456540385080985Worst:  1.0000040011603626
Crossover

Best:  0.5196737980620347Average:  0.9456540385080985Worst:  1.0000040011603626
Mutation

Best:  0.5196737980620347Average:  0.9456540385080985Worst:  1.0000040011603626
Mutation

Best:  0.5196737980620347Average:  0.9456540385080985Worst:  1.0000040011603626
Mutation

Best:  0.5196737980620347Average:  0.9456540385080985Worst:  1.0000040011603626
Crossover

Best:  0.5196737980620347Average:  0.9456540385080985Worst:  1.0000040011603626
Mutation

Best:  0.5196737980620347Average:  0.9456540385080985Worst:  1.0000040011603626
Crossover

Best:  0.5196737980620347Average:  0.9456540385080985Worst:  1.0000040011603626
Crossover

Upd

Best:  0.5196737980620347Average:  0.9456540385080985Worst:  1.0000040011603626
Crossover

Best:  0.5196737980620347Average:  0.9456540385080985Worst:  1.0000040011603626
Crossover

Best:  0.5196737980620347Average:  0.9456540385080985Worst:  1.0000040011603626
Crossover

Best:  0.5196737980620347Average:  0.9456540385080985Worst:  1.0000040011603626
Crossover

Best:  0.5196737980620347Average:  0.9456540385080985Worst:  1.0000040011603626
Crossover

Best:  0.5196737980620347Average:  0.9456540385080985Worst:  1.0000040011603626
Crossover

Best:  0.5196737980620347Average:  0.9456540385080985Worst:  1.0000040011603626
Crossover

Update!!
Best:  0.5196737980620347Average:  0.9456540385080985Worst:  1.0000040011603626
Crossover

Best:  0.5196737980620347Average:  0.9456540385080985Worst:  1.0000040011603626
Crossover

Best:  0.5196737980620347Average:  0.9456540385080985Worst:  1.0000040011603626
Crossover

Best:  0.5196737980620347Average:  0.9456540385080985Worst:  1.0000040011603626
C

Best:  0.5196737980620347Average:  0.9456540385080985Worst:  1.0000040011603626
Mutation

Best:  0.5196737980620347Average:  0.9456540385080985Worst:  1.0000040011603626
Crossover

Best:  0.5196737980620347Average:  0.9456540385080985Worst:  1.0000040011603626
Crossover

Best:  0.5196737980620347Average:  0.9456540385080985Worst:  1.0000040011603626
Crossover

Best:  0.5196737980620347Average:  0.9456540385080985Worst:  1.0000040011603626
Crossover

Best:  0.5196737980620347Average:  0.9456540385080985Worst:  1.0000040011603626
Crossover

Best:  0.5196737980620347Average:  0.9456540385080985Worst:  1.0000040011603626
Crossover

Best:  0.5196737980620347Average:  0.9456540385080985Worst:  1.0000040011603626
Crossover

Best:  0.5196737980620347Average:  0.9456540385080985Worst:  1.0000040011603626
Crossover

Update!!
Best:  0.5196737980620347Average:  0.9456540385080985Worst:  1.0000040011603626
Crossover

Best:  0.5196737980620347Average:  0.9456540385080985Worst:  1.0000040011603626
Cr

In [29]:
solution1.iteration(elitist_operators=True)

Crossover

Best:  0.5196737980620347Average:  0.9427798359160756Worst:  1.0000013338284977


In [30]:
solution1.population[0].show()

0 :  *
1 :  -
2 :  4.9404721059493015
2 :  *
3 :  x1
3 :  0.7035165858253534
1 :  +
2 :  *
3 :  x1
3 :  -
4 :  x1
4 :  1.9937675120699216
2 :  *
3 :  x1
3 :  0.9769954457399338


In [31]:
solution1.population[0].show()

0 :  *
1 :  -
2 :  4.9404721059493015
2 :  *
3 :  x1
3 :  0.7035165858253534
1 :  +
2 :  *
3 :  x1
3 :  -
4 :  x1
4 :  1.9937675120699216
2 :  *
3 :  x1
3 :  0.9769954457399338


In [32]:
solution1.population[1].show()

0 :  -
1 :  *
2 :  x1
2 :  +
3 :  *
4 :  x1
4 :  4.895559976145223
3 :  x2
1 :  *
2 :  +
3 :  x2
3 :  x2
2 :  x1
